論文  
https://arxiv.org/abs/2112.01573  
  
GitHub  
https://github.com/gnobitab/FuseDream  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/FuseDream_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

In [ ]:
!nvidia-smi

# GitHubからFuseDreamのソースコードを取得

In [ ]:
%cd /content/

!git clone https://github.com/gnobitab/FuseDream.git

# ライブラリのインストール

In [ ]:
!pip install ftfy regex tqdm numpy scipy h5py lpips==0.1.4
!pip install git+https://github.com/openai/CLIP.git
!pip install gdown

# 学習済みモデルのダウンロード
  
biggan-256.pth, biggan-512.pthをダウンロード

In [ ]:
%cd /content/FuseDream/BigGAN_utils/weights/

!gdown 'https://drive.google.com/uc?id=17ymX6rhsgHDZw_g5XgAFW4xLSDocARCM'
!gdown 'https://drive.google.com/uc?id=1sOZ9og9kJLsqMNhaDnPJgzVsBZQ1sjZ5'

# ライブラリのインポート

In [ ]:
%cd /content/FuseDream/

import torch
from tqdm import tqdm
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torchvision
import BigGAN_utils.utils as utils
import clip
import torch.nn.functional as F
from DiffAugment_pytorch import DiffAugment
import numpy as np
from fusedream_utils import FuseDreamBaseGenerator, get_G, save_image

import os
import sys
from IPython import display

# パラメータ設定
SENTENCE  
画像を生成するためのクエリテキスト  
文の終わりに明示的にピリオド「.」を付けることで画像の品質を高めることができます。  
  
INIT_ITERS  
初期化に使用する画像の数  
論文中ではMと記載。M=INIT_ITERS*10。Default 1000  
  
OPT_ITERS  
潜在変数を最適化するための反復回数  
Default 1000  
  
NUM_BASIS  
最適化で使用される基底画像  
5, 10, 15から選択  
  
MODEL  
使用するモデル  
bigggan-256, biggan-512から選択  
  
SEED  
ランダムシード  

In [ ]:
#@title パラメータ設定
SENTENCE = "A photo of winged dog." #@param {type:"string"}
INIT_ITERS =  1000 #@param {type:"number"}
OPT_ITERS = 1000 #@param {type:"number"}
NUM_BASIS = 10 #@param {type:"slider", min:0, max:15, step:1}
MODEL = "biggan-256" #@param ["biggan-256","biggan-512"]
SEED =  123#@param {type:"number"}

sys.argv = [''] ### workaround to deal with the argparse in Jupyter

# 画像生成

In [ ]:
%%time

%cd /content/FuseDream/

# 生成画像出力先
if not os.path.exists('./results'):
    os.mkdir('./results')

# ランダムシード設定
utils.seed_rng(SEED) 

sentence = SENTENCE

print('Generating:', sentence)
# Modelのビルド
if MODEL == "biggan-256":
    G, config = get_G(256) 
elif MODEL == "biggan-512":
    G, config = get_G(512) 
else:
    raise Exception('Model not supported')

# 画像生成
generator = FuseDreamBaseGenerator(G, config, 10) 
z_cllt, y_cllt = generator.generate_basis(sentence, init_iters=INIT_ITERS, num_basis=NUM_BASIS)

z_cllt_save = torch.cat(z_cllt).cpu().numpy()
y_cllt_save = torch.cat(y_cllt).cpu().numpy()
# latent_noise=Trueの場合AugCLIPが高くなりますが、画質がわずかに低くなります。
img, z, y = generator.optimize_clip_score(z_cllt, y_cllt, sentence, latent_noise=False, augment=True, opt_iters=OPT_ITERS, optimize_y=True)

score = generator.measureAugCLIP(z, y, sentence, augment=True, num_samples=20)
print('AugCLIP score:', score)

result_file_name = 'results/fusedream_%s_seed_%d_score_%.4f.png'%(sentence, SEED, score)
save_image(img, result_file_name)



# 生成結果の表示

In [ ]:
display.display( display.Image(result_file_name) )

# BigSleepとの比較

## BigSleepのセットアップ

In [ ]:
!pip install big-sleep --upgrade

## ライブラリのインポート

In [ ]:
from tqdm.notebook import trange
from IPython.display import Image, display

from big_sleep import Imagine

## パラメータ設定

In [ ]:
TEXT = SENTENCE
SAVE_EVERY =  100 #@param {type:"number"}
SAVE_PROGRESS = True 
LEARNING_RATE = 5e-2 
ITERATIONS = 1000 #@param {type:"number"}
SEED = SEED

In [ ]:
%%time
%cd /content/FuseDream/

model = Imagine(
    text = TEXT,
    save_every = SAVE_EVERY,
    lr = LEARNING_RATE,
    iterations = ITERATIONS,
    save_progress = SAVE_PROGRESS,
    seed = SEED
)

for epoch in trange(20, desc = 'epochs'):
    for i in trange(1000, desc = 'iteration'):
        model.train_step(epoch, i)

        if i == 0 or i % model.save_every != 0:
            continue

        filename = TEXT.replace(' ', '_')
        image = Image(f'./{filename}.png')
        display(image)